In [1]:
from daft import serving

In [2]:
client = serving.ServingClient()

In [3]:
client.list_backends()

{'default': <daft.serving.backend.DockerEndpointBackend at 0x1053577f0>,
 'aws_lambda': <daft.serving.backend.AWSLambdaEndpointBackend at 0x10f40c670>}

In [4]:
import numpy as np

def myfunc():
    return np.ones((10, 10)).mean().item()

In [5]:
client.list_endpoints()

Backend: default
Backend: aws_lambda
  Endpoint(name='daft-serving-bar', version='2', addr=None)
  Endpoint(name='daft-serving-foo', version='1', addr='https://5q6zx74vtdhwrsm7imnhuavjne0cqpre.lambda-url.us-west-2.on.aws/')
  Endpoint(name='daft-serving-baz', version='4', addr='https://wjkmqmwrux7nlqarq4fsx7lzxa0eabbr.lambda-url.us-west-2.on.aws/')


In [7]:
client.deploy("docker_endpoint_myfunc", myfunc, backend="default", pip_dependencies=["numpy"])
client.deploy("lambda_endpoint_myfunc", myfunc, backend="aws_lambda", pip_dependencies=["numpy"])

DaFt is building your server
Your server was built successfully!
Started endpoint docker_endpoint_myfunc/v0 at http://localhost:52703
  Using cached cloudpickle-2.1.0-py3-none-any.whl (25 kB)
  Using cached numpy-1.19.5-cp39-cp39-manylinux1_x86_64.whl (13.4 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas 1.4.3 requires numpy>=1.20.0; platform_machine == "arm64" and python_version < "3.10", but you have numpy 1.19.5 which is incompatible.
icebridge 0.0.3 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
daft 0.1.0 requires numpy<2.0.0,>=1.22.3, but you have numpy 1.19.5 which is incompatible.
You should consider upgrading via the '/Users/jaychia/Library/Caches/pypoetry/virtualenvs/daft-5WWk1bo2-py3.9/bin/python -m pip install --upgrade pip' command.


Endpoint(name='lambda_endpoint_myfunc', version=1, addr='https://e7y3cpyedyrerq4rzvawahretq0quieo.lambda-url.us-west-2.on.aws/')

In [8]:
client.list_endpoints()

Backend: default
  Endpoint(name='docker_endpoint_myfunc', version=0, addr='http://localhost:52703')
Backend: aws_lambda
  Endpoint(name='daft-serving-bar', version='2', addr=None)
  Endpoint(name='daft-serving-foo', version='1', addr='https://5q6zx74vtdhwrsm7imnhuavjne0cqpre.lambda-url.us-west-2.on.aws/')
  Endpoint(name='daft-serving-lambda_endpoint_myfunc', version='1', addr='https://e7y3cpyedyrerq4rzvawahretq0quieo.lambda-url.us-west-2.on.aws/')
  Endpoint(name='daft-serving-baz', version='4', addr='https://wjkmqmwrux7nlqarq4fsx7lzxa0eabbr.lambda-url.us-west-2.on.aws/')


In [10]:
import requests

In [11]:
response = requests.get('http://localhost:52703')
response.text

'1.0'

In [12]:
response = requests.get('https://e7y3cpyedyrerq4rzvawahretq0quieo.lambda-url.us-west-2.on.aws/')
response.text

'1.0'